In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, RNN, SimpleRNNCell
from tensorflow.keras.models import Sequential
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [2]:
class CustomRNNCell(SimpleRNNCell):
    def __init__(self, units, **kwargs):
        super().__init__(units, **kwargs)
        # Additional parameters for IIR filter simulation
        self.kernel_iir = self.add_weight(shape=(units, units), initializer='uniform', trainable=True) 
        #weights of an IIR filter which will also change while training
    def call(self, inputs, states):
        prev_output = states[0]
        output = tf.matmul(inputs, self.kernel) + tf.matmul(prev_output, self.recurrent_kernel)
        # Simulate the effect of an IIR filter
        output += tf.matmul(prev_output, self.kernel_iir)
        output = tf.nn.relu(output)  # non-linear activation, can be changed to another function
        return output, [output]

In [3]:
class DataPreprocessor:
    def __init__(self, filepath):
        self.filepath = filepath
        self.label_encoder = LabelEncoder()
    
    def load_data(self):
        self.data = pd.read_excel(self.filepath)
        self.features = self.data.iloc[:, :-1]
        self.labels = self.data.iloc[:, -1]
    
    def clean_data(self):
        self.features = self.features.apply(pd.to_numeric, errors='coerce')
        self.features = self.features.fillna(self.features.mean())
    
    def encode_labels(self):
        self.labels = self.labels.astype(str)
        self.labels = self.label_encoder.fit_transform(self.labels)
    
    def scale_features(self):
        scaler = StandardScaler()
        self.features = scaler.fit_transform(self.features)
    
    def split_data(self, test_size=0.2, random_state=42):
        return train_test_split(self.features, self.labels, test_size=test_size, random_state=random_state)
    
    def preprocess(self):
        self.load_data()
        self.clean_data()
        self.encode_labels()
        self.scale_features()
        return self.split_data()

In [4]:
class RNNFaultModel:
    def __init__(self, num_conditions):
        self.model = Sequential([
            RNN(CustomRNNCell(50), return_sequences=True, input_shape=(None, 1)),
            RNN(CustomRNNCell(50)),
            Dense(num_conditions, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    def train(self, X_train, y_train, epochs=10):
        self.model.fit(X_train, y_train, epochs=epochs)
    
    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)
    
    def predict_probabilities(self, X):
        return self.model.predict(X)


In [5]:
class DynamicBayesianNetwork:
    def __init__(self, conditions):
        self.conditions = conditions
        self.model = BayesianModel()
        self.setup_model()
    
    def setup_model(self):
        self.model.add_nodes_from(['State'] + self.conditions)
        for condition in self.conditions:
            self.model.add_edge('State', condition)
        cpd_state = TabularCPD(variable='State', variable_card=2, values=[[0.5], [0.5]])
        for condition in self.conditions:
            cpd_fault = TabularCPD(variable=condition, variable_card=2,
                                   values=[[0.95, 0.05], [0.05, 0.95]],
                                   evidence=['State'], evidence_card=[2])
            self.model.add_cpds(cpd_fault)
        self.model.add_cpds(cpd_state)
        assert self.model.check_model(), "Model has errors"
        self.inference = VariableElimination(self.model)
    
    def update_probabilities(self, evidence):
        # Avoid querying variables that are already part of the evidence
        evidence_keys = set(evidence.keys())
        query_vars = [var for var in self.conditions if var not in evidence_keys]
        updated_beliefs = self.inference.query(variables=query_vars, evidence=evidence)
        return updated_beliefs
    
    def get_most_probable_fault(self, evidence):
        updated_beliefs = self.update_probabilities(evidence)
        max_prob = 0
        most_probable_fault = None
        if isinstance(updated_beliefs, dict):
            for condition, factor in updated_beliefs.items():
                prob_of_fault = factor.values[1]
                if prob_of_fault > max_prob:
                    max_prob = prob_of_fault
                    most_probable_fault = condition
        return most_probable_fault, max_prob

In [6]:
class ModelEvaluator:
    def __init__(self, model, X_test, y_test):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
    
    def predict_random_samples(self, n_samples):
        idx = np.random.choice(self.X_test.shape[0], n_samples, replace=False)
        predictions = self.model.predict_probabilities(self.X_test[idx])
        predicted_classes = np.argmax(predictions, axis=1)
        accuracy = accuracy_score(self.y_test[idx], predicted_classes)
        return predictions, accuracy

In [7]:
class ResidualCalculator:
    def __init__(self, model):
        self.model = model
    
    def calculate_residuals(self, X_test, y_test):
        predictions = self.model.predict_probabilities(X_test)
        predicted_classes = np.argmax(predictions, axis=1)
        residuals = np.abs(y_test - predicted_classes)
        return residuals

In [8]:
def main():
    file_path = r'C:\Users\SIDDHARTH SINGH\Downloads\Classification_of_Faults\Merged_dataset_after_Applying_Hurst_Exponent.xlsx'
    data_preprocessor = DataPreprocessor(file_path)
    X_train, X_test, y_train, y_test = data_preprocessor.preprocess()
    
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    num_conditions = len(np.unique(y_train))
    rnn_fault_model = RNNFaultModel(num_conditions)
    rnn_fault_model.train(X_train, y_train, epochs=8000, learning_rate=0.0001, batch_size=64)
    print("Evaluation on test data:")
    test_loss, test_accuracy = rnn_fault_model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy:.2f}, Test Loss: {test_loss:.2f}")

    # Initialize and utilize DBN
    string_conditions = [str(condition) for condition in np.unique(y_train)]
    dbn = DynamicBayesianNetwork(string_conditions)
    probabilities = rnn_fault_model.predict_probabilities(X_test[0:1])[0]  # Predictions for a single instance
    most_probable_fault, max_prob = dbn.get_most_probable_fault({'State': 1})  # Assuming state 1 is fault
    print(f"The most probable fault is {most_probable_fault} with a probability of {max_prob:.2f}")

    # Additional evaluation on random samples
    evaluator = ModelEvaluator(rnn_fault_model, X_test, y_test)
    n = 1500  # number of random samples to evaluate
    predictions, accuracy = evaluator.predict_random_samples(n)
    print(f"Predictions for {n} random samples: {predictions}")
    print(f"Accuracy on random samples: {accuracy:.2f}")

if __name__ == "__main__":
    main()

Epoch 1/3000


201/201 [==============================] - 3s 6ms/step - loss: 1.2552 - accuracy: 0.5759
Epoch 2/3000
201/201 [==============================] - 1s 6ms/step - loss: 0.4228 - accuracy: 0.8357
Epoch 3/3000
201/201 [==============================] - 1s 6ms/step - loss: 0.3896 - accuracy: 0.8515
Epoch 4/3000
201/201 [==============================] - 1s 6ms/step - loss: 0.3455 - accuracy: 0.8649
Epoch 5/3000
201/201 [==============================] - 1s 5ms/step - loss: 0.3096 - accuracy: 0.8766
Epoch 6/3000
201/201 [==============================] - 1s 6ms/step - loss: 0.2857 - accuracy: 0.8910
Epoch 7/3000
201/201 [==============================] - 1s 6ms/step - loss: 0.2822 - accuracy: 0.8985
Epoch 8/3000
201/201 [==============================] - 1s 5ms/step - loss: 0.2738 - accuracy: 0.8939
Epoch 9/3000
201/201 [==============================] - 1s 5ms/step - loss: 0.2608 - accuracy: 0.9013
Epoch 10/3000
201/201 [==============================] - 1s 5ms/step - loss: 0.2660 - accuracy:

Epoch 159/3000
201/201 [==============================] - 2s 9ms/step - loss: 0.0793 - accuracy: 0.9684
Epoch 160/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0811 - accuracy: 0.9686
Epoch 161/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0785 - accuracy: 0.9678
Epoch 162/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0798 - accuracy: 0.9674
Epoch 163/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0681 - accuracy: 0.9742
Epoch 164/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.1194 - accuracy: 0.9561
Epoch 165/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0877 - accuracy: 0.9681
Epoch 166/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0918 - accuracy: 0.9666
Epoch 167/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.1047 - accuracy: 0.9581
Epoch 168/3000
201/201 [==============================] - 2s 10m

201/201 [==============================] - 2s 8ms/step - loss: 0.0732 - accuracy: 0.9717
Epoch 316/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0806 - accuracy: 0.9689
Epoch 317/3000
201/201 [==============================] - 2s 9ms/step - loss: 0.0686 - accuracy: 0.9719
Epoch 318/3000
201/201 [==============================] - 2s 9ms/step - loss: 0.0611 - accuracy: 0.9761
Epoch 319/3000
201/201 [==============================] - 2s 9ms/step - loss: 0.0526 - accuracy: 0.9788
Epoch 320/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0613 - accuracy: 0.9774
Epoch 321/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0562 - accuracy: 0.9800
Epoch 322/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0795 - accuracy: 0.9689
Epoch 323/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0659 - accuracy: 0.9759
Epoch 324/3000
201/201 [==============================] - 2s 8ms/step - loss: 0

201/201 [==============================] - 2s 10ms/step - loss: 0.0627 - accuracy: 0.9772
Epoch 472/3000
201/201 [==============================] - 2s 10ms/step - loss: 0.0584 - accuracy: 0.9781
Epoch 473/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0448 - accuracy: 0.9830
Epoch 474/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0394 - accuracy: 0.9850
Epoch 475/3000
201/201 [==============================] - 2s 9ms/step - loss: 0.0408 - accuracy: 0.9852
Epoch 476/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0445 - accuracy: 0.9828
Epoch 477/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0580 - accuracy: 0.9755
Epoch 478/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0857 - accuracy: 0.9694
Epoch 479/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.1063 - accuracy: 0.9625
Epoch 480/3000
201/201 [==============================] - 2s 9ms/step - loss:

...

201/201 [==============================] - 2s 9ms/step - loss: 0.0658 - accuracy: 0.9742
Epoch 2811/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0656 - accuracy: 0.9738
Epoch 2812/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0512 - accuracy: 0.9817
Epoch 2813/3000
201/201 [==============================] - 2s 7ms/step - loss: 0.0900 - accuracy: 0.9674
Epoch 2814/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0489 - accuracy: 0.9800
Epoch 2815/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0700 - accuracy: 0.9744
Epoch 2816/3000
201/201 [==============================] - 2s 10ms/step - loss: 0.0477 - accuracy: 0.9792
Epoch 2817/3000
201/201 [==============================] - 2s 10ms/step - loss: 0.0403 - accuracy: 0.9848
Epoch 2818/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0404 - accuracy: 0.9828
Epoch 2819/3000
201/201 [==============================] - 2s 8ms/ste

201/201 [==============================] - 1s 7ms/step - loss: 0.0356 - accuracy: 0.9869
Epoch 2889/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0354 - accuracy: 0.9866
Epoch 2890/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0337 - accuracy: 0.9884
Epoch 2891/3000
201/201 [==============================] - 2s 9ms/step - loss: 0.0542 - accuracy: 0.9786
Epoch 2892/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0325 - accuracy: 0.9869
Epoch 2893/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0308 - accuracy: 0.9880
Epoch 2894/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0395 - accuracy: 0.9833
Epoch 2895/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0336 - accuracy: 0.9864
Epoch 2896/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0669 - accuracy: 0.9783
Epoch 2897/3000
201/201 [==============================] - 1s 7ms/step 

201/201 [==============================] - 2s 8ms/step - loss: 0.0438 - accuracy: 0.9827
Epoch 2967/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0377 - accuracy: 0.9848
Epoch 2968/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0377 - accuracy: 0.9844
Epoch 2969/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0543 - accuracy: 0.9788
Epoch 2970/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0671 - accuracy: 0.9767
Epoch 2971/3000
201/201 [==============================] - 2s 8ms/step - loss: 0.0424 - accuracy: 0.9841
Epoch 2972/3000
201/201 [==============================] - 2s 7ms/step - loss: 0.0296 - accuracy: 0.9892
Epoch 2973/3000
201/201 [==============================] - 1s 7ms/step - loss: 0.0277 - accuracy: 0.9895
Epoch 2974/3000
201/201 [==============================] - 2s 10ms/step - loss: 0.0605 - accuracy: 0.9769
Epoch 2975/3000
201/201 [==============================] - 2s 9ms/step

Test Accuracy: 0.97, Test Loss: 0.13
1/1 [==============================] - 1s 592ms/step


The most probable fault is None with a probability of 0.00
47/47 [==============================] - 0s 4ms/step
Predictions for 1500 random samples: [[0.0000000e+00 4.9538512e-32 2.3632133e-25 ... 1.0754705e-29
  0.0000000e+00 0.0000000e+00]
 [1.1546430e-30 9.0811834e-28 1.7971173e-33 ... 2.0246104e-27
  1.6369505e-13 5.5065425e-29]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.2079648e-27
  4.3509914e-21 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 4.0909673e-18
  0.0000000e+00 0.0000000e+00]
 [9.1221351e-24 5.3529904e-20 4.5811572e-26 ... 2.4123738e-30
  9.9733692e-01 8.1005211e-26]
 [9.2041318e-33 2.2554808e-13 5.8992448e-26 ... 1.0000000e+00
  3.2723911e-36 0.0000000e+00]]
Accuracy on random samples: 0.96
